In [1]:
import numpy, scipy, matplotlib.pyplot as plt, pandas, librosa
from IPython.display import Audio
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from os import path
import sys
import random
from random import randint

In [2]:
slicelength = 50
feature_power = 2 #TODO play with this
data_proportion = .01

classical_names = [f for f in os.listdir("./classical_short_%i/" % slicelength)]
country_names = [f for f in os.listdir("./country_short_%i/" % slicelength)]

num_datapoints = int(min(len(classical_names), len(country_names)) * data_proportion)
print "num datapoints: " + str(num_datapoints)

random.shuffle(classical_names)
random.shuffle(country_names)
classical_names = classical_names[0:num_datapoints]
country_names = country_names[0:num_datapoints]

def stft_array(names, kind):
    update_per = len(names)/10
    print "update per %i" % update_per
    array = []
    for i, x in enumerate(names):
        song_path = '%s_short_%i/%s' % (kind, slicelength, x)
        if i%update_per == 0:
            sys.stdout.write(str(i/update_per) + " ")
            print "adding " + song_path + " to " + kind
        csong_stft = librosa.stft(librosa.load(song_path)[0])
        csong_log_stft = librosa.logamplitude(np.abs(csong_stft)**feature_power,ref_power=np.max)
        csong_log_stft = [abs(item)/80.0 for sublist in csong_log_stft for item in sublist] #TODO: consider removing abs / 80
        array.append(csong_log_stft)
    return array

#shape is 4 by 1025
classical_train = stft_array(classical_names, 'classical')
country_train = stft_array(country_names, 'country')

test_classical = []
test_country = []

print "number of classical slices: %i" % len(classical_train)

while len(test_classical) < len(classical_train) / 10:
    r = randint(0, len(classical_train) - 1)
    test_classical.append(classical_train[r])
    del classical_train[r]
    
print "number of classical training slices: %i" % len(classical_train)
print "number of classical testing slices: %i" % len(test_classical)

print "number of country slices: %i" % len(country_train)
while len(test_country) < len(country_train) / 10:
    r = randint(0, len(country_train) - 1)
    test_country.append(country_train[r])
    del country_train[r]
    
print "number of country training slices: %i" % len(country_train)
print "number of country testing slices: %i" % len(test_country)

num datapoints: 408
update per 40
0 adding classical_short_50/song3_4257.mp3 to classical
1 adding classical_short_50/song5_1822.mp3 to classical
2 adding classical_short_50/song9_768.mp3 to classical
3 adding classical_short_50/song6_2801.mp3 to classical
4 adding classical_short_50/song3_1971.mp3 to classical
5 adding classical_short_50/song9_3734.mp3 to classical
6 adding classical_short_50/song5_3553.mp3 to classical
7 adding classical_short_50/song6_4625.mp3 to classical
8 adding classical_short_50/song9_4760.mp3 to classical
9 adding classical_short_50/song6_5320.mp3 to classical
10 adding classical_short_50/song0_307.mp3 to classical
update per 40
0 adding country_short_50/song8_3280.mp3 to country
1 adding country_short_50/song9_2724.mp3 to country
2 adding country_short_50/song0_1321.mp3 to country
3 adding country_short_50/song9_1466.mp3 to country
4 adding country_short_50/song6_611.mp3 to country
5 adding country_short_50/song7_2133.mp3 to country
6 adding country_short_50/

In [3]:
testing_with_mnist = False

if testing_with_mnist:
    # testing to see if mnist will work here
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

    training = mnist.train.images
    testing = mnist.test.images
    training_validation_one_hot = mnist.train.labels
    testing_validation_one_hot = mnist.test.labels
else:
    training = (numpy.array(classical_train + country_train)).astype(numpy.float32)
    testing = (numpy.array(test_classical + test_country)).astype(numpy.float32)
    classical_one_hot = numpy.array([0,1])
    country_one_hot = numpy.array([1,0])
    training_validation_one_hot = (numpy.array([classical_one_hot for x in classical_train] + [country_one_hot for x in country_train])).astype(numpy.float32)
    testing_validation_one_hot = (numpy.array([classical_one_hot for x in test_classical] + [country_one_hot for x in test_country])).astype(numpy.float32)

def type_check(array_types, num_types):
    error_flag = False
    for x in array_types:
        if type(x) is numpy.ndarray:
            print "ARRAY TYPE ERROR"
            error_flag = True
    for x in num_types:
        if type(x) is numpy.float32:
            print "NUM TYPE ERROR"
            error_flag = True 
    if not error_flag:
        print "All types check out"
        
array_types = [type(training), type(training[0]), type(training_validation_one_hot), type(training_validation_one_hot[0]),
              type(testing), type(testing[0]), type(testing_validation_one_hot), type(testing_validation_one_hot[0])]
num_types = [type(training[0][0]), type(training_validation_one_hot[0][0]), type(testing[0][0]), type(testing_validation_one_hot[0][0])]
type_check(array_types, num_types)      

if len(training)==len(training_validation_one_hot) and len(testing) == len(testing_validation_one_hot) and len(training[0])==len(testing[0]):
    print "training: " + str(len(training))
    print "testing: " + str(len(testing))
    print "num features: " + str(len(training[0]))
else:
    print "ERROR"

All types check out
training: 742
testing: 74
num features: 4100


In [3]:
# max_array = [max(x) for x in training]
# print "max val: " + str(max(max_array))
# min_array = [min(x) for x in training]
# print "min val: " + str(min(min_array))
# print "peek: " + str(training[10][0:200])

In [4]:
def shuffle_array(xs, ys):
    zipped = zip(xs,ys)
    random.shuffle(zipped)
    return zip(*zipped)
# print training_validation_one_hot[0:10]
type_check(array_types, num_types)

training, training_validation_one_hot = shuffle_array(training, training_validation_one_hot)
testing, testing_validation_one_hot = shuffle_array(testing, testing_validation_one_hot)

# print training_validation_one_hot[0:10]
type_check(array_types, num_types)

# print training[5:10]

All types check out
All types check out


In [5]:
numLabels = len(training_validation_one_hot[0])
featureSize = len(training[0])
print "num labels: "+str(numLabels)
print "num features: " + str(featureSize)

x = tf.placeholder(tf.float32, [None, featureSize])
W = tf.Variable(tf.zeros([featureSize, numLabels]))
b = tf.Variable(tf.zeros([numLabels]))

y = tf.nn.softmax(tf.matmul(x,W) + b)
y_ = tf.placeholder(tf.float32, [None, numLabels])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#convolution and pooling (may not need this)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

#convolution layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#second conv layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#densely connected layer

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#Readout Layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

#Train and Evaluate
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.initialize_all_variables()
sess = tf.InteractiveSession()
sess.run(init)

num labels: 10
num features: 784


In [9]:
training_length = len(training)
batch_size = 50
num_iters = training_length / batch_size

print "num iters: " + str(num_iters)

for i in range(num_iters):
    start = i*batch_size
    end = (i+1)*batch_size
    batch_xs = training[start:end]
    batch_ys = training_validation_one_hot[start:end]
    if i%(num_iters/10) == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch_xs, y_: batch_ys, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i/(num_iters/10), train_accuracy))
    train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})
    
print "\nTraining Complete"
print "Running tests"

print("test accuracy %g"%accuracy.eval(feed_dict={x: testing, y_: testing_validation_one_hot, keep_prob: 1.0}))

# testing = numpy.asarray(testing)
# feed_dict = {x: testing}
# correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# print (sess.run(accuracy, feed_dict={x: testing, y_: testing_validation_one_hot}))

# classification = sess.run(y, {x: testing})
# print classification[0:10]
# print classification.shape
# print testing.shape
# for i in range(10):
#     test = numpy.asarray(classification[i])
#     label = numpy.asarray(testing_validation_one_hot[i])
#     print str(test.argmax()) + " " + str(label.argmax())

num iters: 1100
step 0, training accuracy 0.96
step 1, training accuracy 0.98
step 2, training accuracy 1
step 3, training accuracy 0.98
step 4, training accuracy 0.98
step 5, training accuracy 0.94
step 6, training accuracy 1
step 7, training accuracy 0.98
step 8, training accuracy 0.96
step 9, training accuracy 1

Training Complete
Running tests
test accuracy 0.9754
